In [1]:
import pymongo as py
import os

In [2]:
client = py.MongoClient('mongodb://cosmo-feynman:4rDaE4LwLhwtwKCRIID7BV42Dpz9JOcj3H8kTDUqDmDbkmJiW4HqcGmktibeGWdAkFE5ZE8bQPvR1EncGLSiGA==@cosmo-feynman.documents.azure.com:10255/?ssl=true&replicaSet=globaldb')
db = client.ch


In [3]:
import enchant
import string

def clean_text(pdf_text):
    clean_text_split =[]
    a =[]
    for t in pdf_text:
        lower_case = t.lower()
        lower_case = lower_case.replace("\n", " ")
        translator = str.maketrans('','',string.punctuation)
        a.append(lower_case.translate(translator))
        clean_text_split= [x for x in a if not any(x1.isdigit() for x1 in x)]
    return clean_text_split


def ocr_score(test_text):
    list = clean_text(test_text)
    score1 =0
    score2=0
    positive_ratio =0
    negative_ratio=0
    d = enchant.Dict('en_GB')
    for i in range(0, len(list)):
        if list[i] != '' :
            if d.check(list[i])==True:
                score1= score1 +1
                positive_ratio = score1/len(list)
            elif d.check(list[i])==False:
                score2 =score2 -1
            negative_ratio = score2/len(list)
            
    return {'pos_ratio': positive_ratio, 'neg_ratio':negative_ratio}
   #connects to cosmos db
docs_of_interest = db.docs.find(
        {
            'filetype': 'pdf',
            'doctype': 'annual-returns',
            'extracted_hand': {'$exists': True}
        }
    )
# only scores 55 documents in db
for i, k in enumerate(docs_of_interest):
    if i > 55:
        break
    if k['extracted_hand'] != '':
        text_data =k['extracted_hand']
        clean_data =clean_text(text_data)
        if clean_data != None:
            print('score',i, ocr_score(clean_data))

#returns dictionary to be updated on cosmos db        


score 0 {'neg_ratio': -0.06607929515418502, 'pos_ratio': 0.933920704845815}
score 9 {'neg_ratio': -0.034482758620689655, 'pos_ratio': 0.9655172413793104}
score 10 {'neg_ratio': -0.014705882352941176, 'pos_ratio': 0.9816176470588235}
score 11 {'neg_ratio': -0.017094017094017096, 'pos_ratio': 0.9829059829059829}
score 12 {'neg_ratio': -0.035398230088495575, 'pos_ratio': 0.9646017699115044}
score 13 {'neg_ratio': 0.0, 'pos_ratio': 0.9464285714285714}
score 14 {'neg_ratio': -0.11729253592953176, 'pos_ratio': 0.8799258229021789}
score 15 {'neg_ratio': -0.08808290155440414, 'pos_ratio': 0.9093264248704663}
score 16 {'neg_ratio': -0.05603448275862069, 'pos_ratio': 0.9439655172413793}
score 17 {'neg_ratio': -0.03125, 'pos_ratio': 0.96875}
score 18 {'neg_ratio': -0.07234042553191489, 'pos_ratio': 0.9191489361702128}
score 19 {'neg_ratio': -0.05, 'pos_ratio': 0.9333333333333333}
score 20 {'neg_ratio': -0.05045871559633028, 'pos_ratio': 0.9472477064220184}
score 21 {'neg_ratio': 0, 'pos_ratio': 0